In [ ]:
# imports
import numpy as np
import pandas as pd
import json

In [ ]:
countries_id_df = pd.read_csv('countries_id.csv')
country_map = countries_id_df.set_index('to_country_id')['to_country'].to_dict()

votes_df = pd.read_csv('votes.csv')
votes_df['to_country'] = votes_df['to_country'].map(country_map)
votes_df['from_country'] = votes_df['from_country'].map(country_map)

In [ ]:
sent_votes_df = votes_df[votes_df['round'] == 'final'].drop(columns=['round', 'from_country_id', 'to_country_id'])
sent_votes_df = sent_votes_df.groupby(['year', 'from_country', 'to_country']).sum()
sent_votes_df = sent_votes_df[sent_votes_df['total_points'] != 0]
sent_votes_df['tele_points'] = sent_votes_df['tele_points'].astype(int)
sent_votes_df['jury_points'] = sent_votes_df['jury_points'].astype(int)
sent_votes_df

In [ ]:
result_json = {}

for year, year_df in sent_votes_df.groupby(level=0):
    year_df = year_df.droplevel(0)
    result_json[year] = {}

    for from_country, from_df in year_df.groupby(level=0):
        from_df = from_df.droplevel(0)
        result_json[year][from_country] = from_df.to_dict(orient="index")

In [ ]:
with open("voting_map_data.json", "w") as f:
    json.dump(result_json, f, indent=4)